# Greeks and Sensitivity Analysis

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

## Load the Libs we need

In [1]:
# import Lib
import pandas as pd
import datetime as dt
import pytz
import os
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
import scipy.stats as si
import math
import networkx as nx

# import module
from datetime import datetime, timezone
from datetime import date, time
from math import trunc
from dateutil.parser import parse

## Calculating Greeks Using Python

### Black-Scholes Model

In [2]:
import math
import numpy as np
from scipy.stats import norm

class mb_BlackScholes:
    def __init__(self, S, K, r, T, v, q=0, t=0, option_type='call'):
        self.S = S  # underlying asset price
        self.K = K  # strike price
        self.r = r  # risk-free interest rate
        self.T = T  # time until expiration in years
        self.v = v  # volatility
        self.q = q  # dividend yield
        self.t = t  # current time
        self.option_type = option_type  # option type ('call' or 'put')
        
        # compute d1 and d2
        self.d1 = ((np.log(self.S / self.K) + (self.r - self.q + 0.5 * self.v ** 2) * (self.T - self.t)) /
                  (self.v * np.sqrt(self.T - self.t)))
        self.d2 = self.d1 - self.v * np.sqrt(self.T - self.t)
        
    def price(self):
        if self.option_type == 'call':
            return (self.S * np.exp(-self.q * (self.T - self.t)) * norm.cdf(self.d1) - 
                    self.K * np.exp(-self.r * (self.T - self.t)) * norm.cdf(self.d2))
        elif self.option_type == 'put':
            return (self.K * np.exp(-self.r * (self.T - self.t)) * norm.cdf(-self.d2) - 
                    self.S * np.exp(-self.q * (self.T - self.t)) * norm.cdf(-self.d1))
    
    def delta(self):
        if self.option_type == 'call':
            return np.exp(-self.q * (self.T - self.t)) * norm.cdf(self.d1)
        elif self.option_type == 'put':
            return -np.exp(-self.q * (self.T - self.t)) * norm.cdf(-self.d1)
        
    def gamma(self):
        return ((np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1)) / 
               (self.S * self.v * np.sqrt(self.T - self.t)))
        
    def theta(self):
        if self.option_type == 'call':
            return (-((self.S * np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * self.v) / (2 * np.sqrt(self.T - self.t))) - 
                    self.r * self.K * np.exp(-self.r * (self.T - self.t)) * norm.cdf(self.d2) + 
                    self.q * self.S * np.exp(-self.q * (self.T - self.t)) * norm.cdf(self.d1))
        elif self.option_type == 'put':
            return (-((self.S * np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * self.v) / (2 * np.sqrt(self.T - self.t))) + 
                    self.r * self.K * np.exp(-self.r * (self.T - self.t)) * norm.cdf(-self.d2) - 
                    self.q * self.S * np.exp(-self.q * (self.T - self.t)) * norm.cdf(-self.d1))
        
    def vega(self):
        return self.S * np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * np.sqrt(self.T - self.t)
    
    def rho(self):
        if self.option_type == 'call':
            return self.K * (self.T - self.t) * np.exp(-self.r * (self.T - self.t)) * norm.cdf(self.d2)
        elif self.option_type == 'put':
            return -self.K * (self.T - self.t) * np.exp(-self.r * (self.T - self.t)) * norm.cdf(-self.d2)
        
# Second order Greeks
    
    def charm(self):
        if self.option_type == 'call':
            return -np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * ((2 * (self.r - self.q) * (self.T - self.t) - self.d1 * self.v * np.sqrt(self.T - self.t)) / 
                    (2 * (self.T - self.t) * self.v * np.sqrt(self.T - self.t))) - self.q * np.exp(-self.q * (self.T - self.t)) * norm.cdf(self.d1)
        elif self.option_type == 'put':
            return -np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * ((2 * (self.r - self.q) * (self.T - self.t) - self.d1 * self.v * np.sqrt(self.T - self.t)) / 
                    (2 * (self.T - self.t) * self.v * np.sqrt(self.T - self.t))) + self.q * np.exp(-self.q * (self.T - self.t)) * norm.cdf(-self.d1)
        
    def vomma(self):
        return self.vega() * (self.d1 * self.d2 / self.v)
    
    def vanna(self):
        return -np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) * self.d2 / self.v
        
    def speed(self):
        return np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) / (self.S**2 * self.v * np.sqrt(self.T - self.t)) * (self.d1 + self.v * np.sqrt(self.T - self.t))
        
    def zomma(self):
        return self.gamma() * ((self.d1 * self.d2 - 1) / self.v)
    
    def color(self):
        return np.exp(-self.q * (self.T - self.t)) * norm.pdf(self.d1) / (2 * self.S * self.v * np.sqrt(self.T - self.t)) * (1 + self.d1 * ((2 * (self.r - self.q) * (self.T - self.t) - self.d2 * self.v * np.sqrt(self.T - self.t)) / 
                (2 * self.v * np.sqrt(self.T - self.t))))


In [4]:
# Define option parameters
S = 100  # underlying asset price
K = 105  # strike price
T = 1    # time until expiration in years
r = 0.05 # risk-free interest rate
q = 0.01 # dividend yield
v = 0.20 # volatility

# Calculate the Greeks using the extended BlackScholes class
bs = mb_BlackScholes(S, K, r, T, v, q, option_type='call')

print('Price: ', bs.price())
print('Delta: ', bs.delta())
print('Gamma: ', bs.gamma())
print('Theta: ', bs.theta())
print('Vega: ', bs.vega())
print('Rho: ', bs.rho())

# Calculate and print second-order Greeks
print('Charm: ', bs.charm())
print('Vomma: ', bs.vomma())
print('Vanna: ', bs.vanna())
print('Speed: ', bs.speed())
print('Zomma: ', bs.zomma())
print('Color: ', bs.color())


Price:  7.491693155007894
Delta:  0.5171512290359106
Gamma:  0.019717640994265942
Theta:  -5.637548457246437
Vega:  39.435281988531884
Rho:  44.223429748583165
Charm:  -0.07299050034183298
Vomma:  -1.5908834261433795
Vanna:  0.28383706062941993
Speed:  5.048685791412178e-05
Zomma:  -0.09938364668440139
Color:  0.010009108342042454
